In [21]:
from functools import partial
from pyscf import gto, scf, cc
import numpy as np

from jax import config
config.update("jax_enable_x64", True)

print = partial(print, flush=True)

a = 1.831  #1.05835 # 2aB
d = 3
nH = 20
atoms = ""
for i in range(nH):
    if i % 2 == 0:
        atoms += f"H {i/2*(a+d):.5f} 0.00000 0.00000 \n"
    else:
        atoms += f"H {(i-1)/2*(a+d)+a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="sto6g", unit= "bohr", verbose=4)
mol.build()

mf = scf.RHF(mol).density_fit()
e = mf.kernel()

mycc = cc.CCSD(mf)
mycc.kernel()
et = mycc.ccsd_t()


System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Thu Sep 18 14:48:18 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 20
[INPUT] num. electrons = 20
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.0000

******** <class 'pyscf.df.df.DF'> ********
auxbasis = None
max_memory = 4000
ETB for H: l = 0, exps = 0.200224856 * 2^n , n = 0..8

WARN: Even tempered Gaussians are generated as DF auxbasis for  H

init E= -7.47210711424523
  HOMO = -0.293736394514305  LUMO = 0.164178472445108
cycle= 1 E= -10.7703829260945  delta_E= -3.3  |g|= 0.164  |ddm|= 2.88
  HOMO = -0.407928719025073  LUMO = 0.275670185554826
cycle= 2 E= -10.7887727345425  delta_E= -0.0184  |g|= 0.0453  |ddm|=  0.3
  HOMO = -0.416756410242356  LUMO = 0.287693387525341
cycle= 3 E= -10.7902577898365  delta_E= -0.00149  |g|= 0.000477  |ddm|= 0.112
  HOMO = -0.416513602540557  LUMO = 0.287437543625928
cycle= 4 E= -10.790257946381  delta_E= -1.57e-07  |g|= 7.2e-05  |ddm|= 0.00106
  HOMO = -0.416533572291383  LUMO = 0.287445500630595
cycle= 5 E= -10.7902579498078  delta_E= -3.43e-09  |g|= 7.2e-06  |ddm|= 0.00015
  HOMO = -0.416536205561482  LUMO = 0.287447377512706
cycle= 6 E= -10.790257949845  delta_E= -3.71e-11  |g|= 4.8e-07  |ddm|=

In [27]:
from ebcc import EBCC
myebcc = EBCC(mf)
myebcc = myebcc.kernel()


RDFCCSD
Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0
Ansatz: DFCCSD
Space: (10o, 10v)
Solving for excitation amplitudes.
Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.1968011104     -10.9870590602
   1    -0.2739934334     -11.0642513833     7.719e-02     8.885e-03
   2    -0.3046920336     -11.0949499835     3.070e-02     5.284e-03
   3    -0.3257138535     -11.1159718034     2.102e-02     4.495e-03
   4    -0.3255806330     -11.1158385829     1.332e-04     6.115e-04
   5    -0.3255969565     -11.1158549064     1.632e-05     1.603e-04
   6    -0.3256299460     -11.1158878959     3.299e-05     7.763e-05
   7    -0.3256269467     -11.1158848965     2.999e-06     7.959e-06
   8    -0.3256233362     -11.1158812860     3.610e-06     2.316e-06
   9    -0.3256247059     -11.1158826558     1.370e-06     1.074e-06
  10    -0.3256246919     -11.1158826417     1.403e-08     5.112e-07
  11 

In [ ]:
t1 = mycc.t1
t2 = mycc.t2
myc2 = t2+np.einsum('ia,jb->ijab',t1,t1)-np.einsum('ia,jb->ijba',t1,t1)

In [ ]:
t1 = mycc.t1
t2 = mycc.t2
from ebcc.codegen import GecCC
nocc = np.count_nonzero(mf.mo_occ)
nvir = mol.nao-nocc
t3 = np.empty((nocc,nocc,nocc,nvir,nvir,nvir))
# t4 = np.empty((nocc,nocc,nocc,nocc,nvir,nvir,nvir,nvir))
c1 = GecCC.convert_t1_to_c1(t1)['c1']
c2 = GecCC.convert_t2_to_c2(t1,t2)['c2']
c3 = GecCC.convert_t3_to_c3(t1,t2,t3)['c3']
# c4 = GecCC.convert_t4_to_c4(t1,t2,t3,t4)['c4']

In [34]:
from pyscf import lib
einsum = lib.einsum

In [37]:
nocc = np.count_nonzero(mf.mo_occ)
nvir = mol.nao-nocc
t1 = mycc.t1
t2 = mycc.t2
t3 = np.empty((nocc,nocc,nocc,nvir,nvir,nvir))
c1 = t1
c2 = t2 + einsum('ia,jb->ijab',t1,t1)
c3 = 6*einsum('ia,jkbc->ijkabc',t1,t2) + einsum('ia,jb,kc->ijkabc',t1,t1,t1)

In [ ]:
c3 = np.copy(t3)
c3 += einsum(t2, (0, 1, 2, 3), t1, (4, 5), (0, 4, 1, 5, 2, 3)) * -1
c3 += einsum(t2, (0, 1, 2, 3), t1, (4, 5), (0, 4, 1, 2, 5, 3))
c3 += einsum(t1, (0, 1), t2, (2, 3, 4, 5), (2, 0, 3, 4, 5, 1)) * -1
c3 += einsum(t2, (0, 1, 2, 3), t1, (4, 5), (0, 1, 4, 5, 2, 3))
c3 += einsum(t2, (0, 1, 2, 3), t1, (4, 5), (0, 1, 4, 2, 5, 3)) * -1
c3 += einsum(t2, (0, 1, 2, 3), t1, (4, 5), (0, 1, 4, 2, 3, 5))
tmp0 = np.copy(t2)
tmp0 += einsum(t1, (0, 1), t1, (2, 3), (0, 2, 3, 1)) * -1
tmp0 += einsum(t1, (0, 1), t1, (2, 3), (0, 2, 1, 3))
c3 += einsum(t1, (0, 1), tmp0, (2, 3, 4, 5), (0, 3, 2, 1, 5, 4))
c3 += einsum(t1, (0, 1), tmp0, (2, 3, 4, 5), (0, 3, 2, 5, 4, 1))
c3 += einsum(t1, (0, 1), tmp0, (2, 3, 4, 5), (0, 3, 2, 5, 1, 4)) * -1

In [41]:
o1 = 2*einsum('ia,ia->',c1,c1)
o2 = 2*einsum('ijab,ijab->',c2,c2)-einsum('ijab,ijba->',c2,c2)
o3 = (8*einsum('ijkabc,ijkabc->',c3,c3)-4*(einsum('ijkabc,ijkacb->',c3,c3)
    + einsum('ijkabc,ijkcba->',c3,c3) + einsum('ijkabc,ijkbac->',c3,c3))
    + 2*(einsum('ijkabc,ijkbca->',c3,c3) + einsum('ijkabc,ijkcab->',c3,c3)))/36

In [42]:
print(o1,o2,o3)

0.0004801852641798654 0.27561810935291675 0.00023557396918219602


In [43]:
norm = 1 + o1 + o2 + o3
c = o3/norm
print(norm,c)

1.2763338685862788 0.00018457080469323257


In [ ]:
-10.83072*c - 10.83072

-10.832719034705807

In [81]:
n_c1 = np.sum(c1**2)
n_c2 = np.sum(c2**2)
n_c3 = np.sum(c3**2)
n_c4 = np.sum(c4**2)
print(n_c1,n_c2,n_c3,n_c4)
norm = 1+n_c1+n_c2/4+n_c3/36#+n_c4/(4*3*2)**2
print(norm)

0.0010751417652817646 0.29826462479828164 0.0020933178266828866 1.2182909995542957
1.0756994456822597


In [23]:
def t1t2_2_c1c2c3(mycc):
    from ebcc.codegen import GecCC
    mf = mycc._scf
    t1 = mycc.t1
    t2 = mycc.t2
    nocc = np.count_nonzero(mf.mo_occ)
    nvir = mf.mol.nao-nocc
    t3 = np.empty((nocc,nocc,nocc,nvir,nvir,nvir))
    c1 = GecCC.convert_t1_to_c1(t1)['c1']
    c2 = GecCC.convert_t2_to_c2(t1,t2)['c2']
    c3 = GecCC.convert_t3_to_c3(t1,t2,t3)['c3']
    return c1,c2,c3

In [22]:
def cr_factor(c1,c2,c3,c4=None):
    einsum = lib.einsum
    o1 = 2*einsum('ia,ia->',c1,c1)
    o2 = 2*(4*einsum('ijab,ijab->',c2,c2)-2*einsum('ijab,ijba->',c2,c2))
    o3 = 6*(8*einsum('ijkabc,ijkabc->',c3,c3)-4*(einsum('ijkabc,ijkacb->',c3,c3)
        + einsum('ijkabc,ijkcba->',c3,c3) + einsum('ijkabc,ijkbac->',c3,c3))
        + 2*(einsum('ijkabc,ijkbca->',c3,c3) + einsum('ijkabc,ijkcab->',c3,c3)))
    norm = 1 + o1 + o2/4 + o3/36
    c = o3/36/norm
    # n_c1 = np.sum(c1**2)
    # n_c2 = np.sum(c2**2)
    # n_c3 = np.sum(c3**2)
    # if c4 is None:
    #     factor = (n_c3/36)/(1+n_c1+n_c2/4+n_c3/36)
    # else:
    #     n_c4 = np.sum(c4**2)
    #     factor = (n_c3/36+n_c4/(4*3*2))/(1+n_c1+n_c2/4+n_c3/36+n_c4/(4*3*2))
    return c

def afqmc_cisd_cr(e_afqmc,c):
    cr = e_afqmc*c
    return e_afqmc+cr,cr

In [ ]:
# a=d=1.831 eql
# converged SCF energy = -10.5113174115856
# RCCSD converged
# E(RCCSD) = -10.82386829894422  E_corr = -0.3125508873585868
# DMRG energy =  -10.832557941737981
# AFQMC energy: -10.83072 +/- 0.00009

In [20]:
e_afqmc = -10.83072
e_dmrg = -10.832557941737981
c = cr_factor(c1,c2,c3)
e_afqmc_cr,_ = afqmc_cisd_cr(e_afqmc,c)
print(f'err to DMRG before = {e_afqmc-e_dmrg}, after = {e_afqmc_cr-e_dmrg}')

err to DMRG before = 0.0018379417379819074, after = 0.0014755837202642397


In [ ]:
# a=1.831 d=3
# converged SCF energy = -10.7902563668634
# E_corr(MP2) -0.196801273811774
# E(RCCSD) = -11.11588133074048  E_corr = -0.3256249638770554
# DMRG energy (extrapolated) =  -11.116780958599403 +/- 0.0000000000
# AFQMC energy: -11.11581 +/- 0.00009

In [25]:
e_afqmc = -11.11581
e_dmrg = -11.116780958599403
c = cr_factor(c1,c2,c3)
e_afqmc_cr,_ = afqmc_cisd_cr(-11.11581,c)
print(c)
print('err before = ', e_afqmc-e_dmrg, e_afqmc_cr-e_dmrg)

3.3456503142645046e-05
err before =  0.0009709585994031045 0.0005990624672058686


In [ ]:
# H40 a=d=1.831
# converged SCF energy = -21.0019645737223
# E(RCCSD) = -21.62210834629252  E_corr = -0.6201437725701879
# RCCSD(T) correction = -0.0176994487844547
# ccsd(t) energy: -21.639807795076976
# AFQMC energy: -21.6382 +/- 0.0004
# DMRG energy (extrapolated) =  -21.646735579929590 +/-    0.0000000000

In [ ]:
e_afqmc = -21.6382
e_dmrg = -21.646735579929590
c1,c2,c3 = t1t2_2_c1c2c3(mycc)
c = cr_factor(c1,c2,c3)
e_afqmc_cr,_ = afqmc_cisd_cr(e_afqmc,c)
print(f'err to DMRG before = {e_afqmc-e_dmrg}, after = {e_afqmc_cr-e_dmrg}')

err before =  0.008535579929588977 0.0016923375460038415


In [114]:
print(f'err to DMRG before = {e_afqmc-e_dmrg}, after = {e_afqmc_cr-e_dmrg}')

err to DMRG before = 0.008535579929588977, after = 0.0016923375460038415


In [ ]:
# H40 a=1.831 d=3
# converged SCF energy = -21.5756025423033
# E(RCCSD) = -22.22585047306383  E_corr = -0.6502479307605629
# RCCSD(T) correction = -0.0010780749331425
# ccsd(t) energy: -22.226928547996973
# AFQMC energy: -22.2245 +/- 0.0004
# DMRG energy (extrapolated) =  -22.227757880107031 +/-    0.0000000000

In [4]:
e_afqmc = -22.2245
e_dmrg = -22.227757880107031
c1,c2,c3 = t1t2_2_c1c2c3(mycc)
c = cr_factor(c1,c2,c3)
e_afqmc_cr,_ = afqmc_cisd_cr(e_afqmc,c)
print(f'err to DMRG before = {e_afqmc-e_dmrg}, after = {e_afqmc_cr-e_dmrg}')

err to DMRG before = 0.003257880107032207, after = 0.002019729674799464


In [ ]:
# H80 a = 0.96892 # 1.831aB
# AFQMC energy: -43.2513 +/- 0.0008
# converged SCF energy = -41.9835706086777
# E(RCCSD) = -43.2187515626576
# ccsd(t) energy: -43.25881794557464
# DMRG energy =  -43.277423271142652

In [116]:
e_afqmc = -43.2513
e_dmrg = -43.277423271142652
c1,c2,c3 = t1t2_2_c1c2c3(mycc)
c = cr_factor(c1,c2,c3)
e_afqmc_cr,_ = afqmc_cisd_cr(e_afqmc,c)
print(f'err to DMRG before = {e_afqmc-e_dmrg}, after = {e_afqmc_cr-e_dmrg}')

: 

In [6]:
def davidson_cr(ci_corr,c0,nelec,order=1):
    c2_int_sq = (1-c0**2)/c0**2
    d_cr = c2_int_sq*ci_corr*(1-2/nelec)
    if order == 2:
        d_cr2 = 2*ci_corr*(4/nelec**2-1)*c2_int_sq**2
        d_cr += d_cr2
    return d_cr

In [104]:
nelec = mol.nelectron
c2_int_sq = (1-c0**2)/c0**2
d_cr2 = 2*ci_corr*(4/nelec**2-1)*c2_int_sq**2

In [18]:
nelec = mol.nelectron
d_cr1 = davidson_cr(ci_corr,c0,nelec,1)
d_cr2 = davidson_cr(ci_corr,c0,nelec,2)
print(d_cr1)
print(d_cr2)

-0.04645142872131443
-0.027028322614441352


In [20]:
print(ci_corr)
print(ci_corr+d_cr1)
print(ci_corr+d_cr2)
print(mycc.e_corr+et)

-0.2715561490087084
-0.3180075777300228
-0.2985844716231497
-0.3261313621110479
